# Tensorboard

What?
* visualize your Tensorflow graph
* plot quantative metrics
* show data

Why?
* easier to understand
* easier to debug
* easier to optimize

* Most higher level libraries support it and make using it very effortless:
    * TFLeanr
    * Keras
    * ...
    * Not so much with Edward :/


* Actively developed, more coming:
    * TensorFlow Debugger integration (See if/where there were nans etc)
    * Plugins (people contributing domain specific visualizations)
    * Scalabale TensorBoard (share withing research group)
    
Why not?
* Not so much documentation
* Not many examples
* Almost all examples contributed by other people do not work with current version



## Usage

Use [summary operations](https://www.tensorflow.org/versions/r0.11/api_docs/python/train/#summary_operations) to collect summary data from graph nodes. Summary operations are also given a tag / name that will be used to organize the data in the frontend.
>tf.summary.tensor_summary <br>
>tf.summary.scalar <br>
>tf.summary.histogram <br>
>tf.summary.audio <br>
>tf.summary.image

Then merge all your summary operations in to one:
> tf.summary.merge_all 

Write all summary data to disk using FileWriter. 
> file_writer = tf.summary.FileWriter('/path/to/logs', sess.graph)

When running tensorboard, it recursively walks the directory given to it looking for subdirectories that contain tfevents data. It will visualize all these thus making comparison between different runs easier. You can even give several directories separated with commas.
Run tensorboard:
> tensorboard --logdir=path/to/log-directory --port=6006


## MNIST Examples

### Visualizing tensorflow graph: simple example

In [ ]:
# Simple model for classifying mnist data
# Adapted from: http://ischlag.github.io/2016/06/04/how-to-use-tensorboard/

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# reset everything to rerun in jupyter
tf.reset_default_graph()

batch_size = 100
learning_rate = 0.5
training_epochs = 20
logs_path = "tblogs/reg/"


# Placeholders for inputs
x = tf.placeholder(tf.float32, shape=[None, 784]) 
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# Parameters
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x,W) + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #
    # Write graph using FileWriter
    #
    writer = tf.summary.FileWriter(logs_path)
    writer.add_graph(sess.graph)
        
    for epoch in range(training_epochs):
        batch_count = int(mnist.train.num_examples/batch_size)
        for i in range(batch_count):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            train = sess.run(train_op, feed_dict={x: batch_x, y_: batch_y})

        if epoch % 5 == 0: 
            print("Epoch: ", epoch)
    print("Accuracy: ", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    print("done")

### Adding name scopes and summary operations

* Adding name scopes helps understanding the graph. 
    * Graphs can quickly get complicated with thousands of nodes that can not be visualized at once. Name scopes can be used to define a hierarchy on the nodes which tensorboard then utilizes in the visualization.
* Adding summaries to visualize input data and results. 
    * To visualize the summary data in TensorBoard, you should:
        * evaluate the summary op
        * retrieve the result
        * write that result to disk using a summary.FileWriter
    * Give tags to organise the summary data in the frontend. The scalar and histogram dashboards organize data by tag, and group the tags into folders according to a directory/like/hierarchy

In [ ]:
# Simple model for classifying mnist data
# Adapted from: http://ischlag.github.io/2016/06/04/how-to-use-tensorboard/
# and https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/examples/tutorials/mnist/mnist_with_summaries.py


# reset everything to rerun in jupyter
tf.reset_default_graph()

batch_size = 100
learning_rate = 0.5
training_epochs = 20
logs_path = "tblogs/reg_names/"

#
# Add name scopes to define hierarchy on the nodes
#

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x-input")
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")

with tf.name_scope("weights"):
    W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.1))
    #W = tf.Variable(tf.zeros([784, 10]))

with tf.name_scope("biases"):
    b = tf.Variable(tf.constant(0.1, shape=[10]))
    #b = tf.Variable(tf.zeros([10]))

with tf.name_scope("softmax"):
    y = tf.nn.softmax(tf.matmul(x,W) + b)

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

with tf.name_scope('train'):
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#    
# Create a summaries and merge them
#
tf.summary.scalar("cost", cross_entropy)
tf.summary.scalar("accuracy", accuracy)
tf.summary.histogram("weights", W)
tf.summary.histogram("biases", b)
summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    writer = tf.summary.FileWriter(logs_path)
    writer.add_graph(sess.graph)
        
    for epoch in range(training_epochs):
        batch_count = int(mnist.train.num_examples/batch_size)
        
        for i in range(batch_count):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, summary = sess.run([train_op, summary_op], feed_dict={x: batch_x, y_: batch_y})       
            # write summary log
            writer.add_summary(summary, epoch * batch_count + i)
            
        if epoch % 5 == 0: 
            print("Epoch: ", epoch)
            
    print("Accuracy: ", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    print("done")

### Convolutional neural networks made easy with tflearn

TFLearn automatically supports TensorBoard. You can control what summaries you need with the verbose argument for DNN model class:
>model = DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_path)

where the options are:
* 0: Loss & Metric (Best speed).
* 1: Loss, Metric & Gradients.
* 2: Loss, Metric, Gradients & Weights.
* 3: Loss, Metric, Gradients, Weights, Activations & Sparsity (Best Visualization).


### Comparing different executions of the model

* Changing hyperparameters and comparing results visually
* Tensorboard recursively walks the directory tree rooted at logdir looking for subdirectories that contain tfevents data.



In [ ]:
# Adapted from https://github.com/tflearn/tflearn/blob/master/examples/images/convnet_mnist.py

from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression


def make_modelid(learning_rate, two_conv_layers, two_fc_layers):
    return 'cnn_lr:{0}_conv:{1}_fc:{2}'.format(learning_rate, two_conv_layers, two_fc_layers)

# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

batch_size = 100
training_epochs = 5
logs_path = "tblogs_multiple"

for learning_rate in [0.1, 0.01, 0.001]:
    for two_conv_layers in [True, False]:
        for two_fc_layers in [True, False]:
            tf.reset_default_graph()

            network = input_data(shape=[None, 28, 28, 1], name='input')
            network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
            network = max_pool_2d(network, 2)
            if (two_conv_layers):
                network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
                network = max_pool_2d(network, 2)

            network = fully_connected(network, 128, activation='tanh')
            if (two_fc_layers):
                network = fully_connected(network, 256, activation='tanh')
                
            network = fully_connected(network, 10, activation='softmax')
            network = regression(network, optimizer='adam', learning_rate=learning_rate,
                                 loss='categorical_crossentropy', name='target')

            # Training
            model = tflearn.DNN(network, tensorboard_verbose=2, tensorboard_dir=logs_path)
            model_id = make_modelid(learning_rate, two_conv_layers, two_fc_layers)
            model.fit(X, Y, n_epoch=training_epochs, shuffle=True, show_metric=True, batch_size=batch_size,
                        snapshot_epoch=True, run_id=model_id,
                        validation_set=(testX, testY))

## Edward and TensorBoard
* Visualizing graph possible. Initialize takes log directory as a parameter.

>initialize(logdir='tblogs')

* [Summaries](https://github.com/blei-lab/edward/issues/478) don't really work, but tensorboard will be used for more visualizations [later](https://github.com/blei-lab/edward/issues/190) (?)


In [ ]:
# Copied from GitHub examples: edward/examples/normal_normal_tensorboard.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Normal

ed.set_seed(42)

# DATA
x_data = np.array([0.0] * 50)

# MODEL: Normal-Normal with known variance
mu = Normal(loc=0.0, scale=1.0, name='mu')
x = Normal(loc=tf.ones(50) * mu, scale=1.0, name='x')

# INFERENCE
qmu_mu = tf.Variable(tf.random_normal([]), name='qmu_mu')
qmu_sigma = tf.nn.softplus(tf.Variable(tf.random_normal([])), name='qmu_sigma')
qmu = Normal(loc=qmu_mu, scale=qmu_sigma, name='qmu')

# analytic solution: N(loc=0.0, scale=\sqrt{1/51}=0.140)
inference = ed.KLqp({mu: qmu}, data={x: x_data})
inference.run(logdir='tblogs_edward')

## Embedding Visualization
* Interactive visualization and analysis of high-dimensional data
* Two- or three-dimensional view using
    * PCA: The Embedding Projector computes the top 10 principal components. The menu lets you project those components onto any combination of two or three.
    * t-SNE: Layout is performed client-side animating every step of the algorithm. 
    * Custom

In [ ]:
# Example from https://github.com/anujshah1003/Tensorboard-examples/blob/master/mnist-tensorboard-embeddings-1.py

import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np

PATH = os.getcwd()

LOG_DIR = PATH + '/mnist-tensorboard/log-1'
metadata = os.path.join(LOG_DIR, 'metadata.tsv')
mnist = input_data.read_data_sets(PATH + "/mnist-tensorboard/data/", one_hot=True)

# Setup a 2D tensor that holds your embedding(s)
images = tf.Variable(mnist.test.images, name='images')

# Create metadata
with open(metadata, 'w') as metadata_file:
    for row in range(10000):
        c = np.nonzero(mnist.test.labels[::1])[1:][0][row]
        metadata_file.write('{}\n'.format(c))



with tf.Session() as sess:
    # Save your model variables in a checkpoint in LOG_DIR.
    saver = tf.train.Saver([images])
    sess.run(images.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images.ckpt'))

    config = projector.ProjectorConfig()
    # Add embedding(s)
    embedding = config.embeddings.add()
    embedding.tensor_name = images.name
    # Link to metadata
    embedding.metadata_path = metadata
    # Write a projector_config.pbtxt in the LOG_DIR. TensorBoard will
    # read this file during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)